# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [1]:
import pandas as pd
artwork = pd.read_csv("./data/Artworks.csv")
import sqlalchemy
sqlalchemy.__version__

'1.2.7'

In [2]:
pd.__version__

'0.24.0'

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [3]:
import pandas as pd
managers = pd.read_csv('~/Desktop/baseball/core/Managers.csv')
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


In [4]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

**Question:** Is there a candidate for a primary key?

In [5]:
[(col, managers[col].is_unique) for col in managers]

[('playerID', False),
 ('yearID', False),
 ('teamID', False),
 ('lgID', False),
 ('inseason', False),
 ('G', False),
 ('W', False),
 ('L', False),
 ('rank', False),
 ('plyrMgr', False)]

**Solution:** Add the `index` as an actual column

In [6]:
from dfply import mutate
managers = (managers >>
            mutate(id = managers.index))

In [7]:
managers.id.is_unique

True

In [8]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr', 'id'],
      dtype='object')

In [9]:
managers.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
id            int64
dtype: object

In [10]:
managers.shape

(3469, 11)

In [11]:
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr,id
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y,0
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y,1
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y,2
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y,3
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y,4


#### Task 2 - Create a `sqlalchemy` types `dict`

In [12]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'plyrMgr':String,
             'teamID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'inseason':Integer, 
             'G':Integer, 
             'W':Integer, 
             'L':Integer,
             'rank':Integer}

#### Task 4 - Create an `engine` and `schema`

In [11]:
!rm databases/baseball.db

rm: cannot remove 'databases/baseball.db': No such file or directory


In [13]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)




#### Execute the `schema`

In [14]:
mang_eng.execute(schema)

2019-01-30 12:30:48,988 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-30 12:30:48,989 INFO sqlalchemy.engine.base.Engine ()
2019-01-30 12:30:48,992 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-30 12:30:48,993 INFO sqlalchemy.engine.base.Engine ()
2019-01-30 12:30:48,995 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)


2019-01-30 12:30:48,996 INFO sqlalchemy.engine.base.Engine ()
2019-01-30 12:30:49,017 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
managers.shape

(3469, 11)

#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [16]:
managers.to_sql('manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-30 12:30:54,238 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("manager")
2019-01-30 12:30:54,239 INFO sqlalchemy.engine.base.Engine ()
2019-01-30 12:30:54,246 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-30 12:30:54,377 INFO sqlalchemy.engine.base.Engine INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-01-30 12:30:54,379 INFO sqlalchemy.engine.base.Engine (('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y', 0), ('woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y', 1), ('paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y', 2), ('lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y', 3), ('deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y', 4), ('fergubo01', 1871, 'NY2', None, 1, 33, 16, 17, 5.0, 'Y', 5), ('mcbridi01', 1871, 'PH1', None, 1, 28, 21, 7, 1.0, 'Y', 6), ('hastisc01', 1871, 'RC1', None, 1, 25, 4, 21, 9.0, 'Y', 7)  ... displaying 10 of

#### Task 6 - Query the table to make sure it all worked

In [17]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [18]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.manager

In [19]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-30 12:31:00,422 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-30 12:31:00,424 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM manager
2019-01-30 12:31:00,427 INFO sqlalchemy.engine.base.Engine ()


[{'playerID': 'wrighha01',
  'yearID': 1871,
  'teamID': 'BS1',
  'lgID': None,
  'inseason': 1,
  'G': 31,
  'W': 20,
  'L': 10,
  'rank': 3,
  'plyrMgr': 'Y',
  'id': 0},
 {'playerID': 'woodji01',
  'yearID': 1871,
  'teamID': 'CH1',
  'lgID': None,
  'inseason': 1,
  'G': 28,
  'W': 19,
  'L': 9,
  'rank': 2,
  'plyrMgr': 'Y',
  'id': 1},
 {'playerID': 'paborch01',
  'yearID': 1871,
  'teamID': 'CL1',
  'lgID': None,
  'inseason': 1,
  'G': 29,
  'W': 10,
  'L': 19,
  'rank': 8,
  'plyrMgr': 'Y',
  'id': 2},
 {'playerID': 'lennobi01',
  'yearID': 1871,
  'teamID': 'FW1',
  'lgID': None,
  'inseason': 1,
  'G': 14,
  'W': 5,
  'L': 9,
  'rank': 8,
  'plyrMgr': 'Y',
  'id': 3},
 {'playerID': 'deaneha01',
  'yearID': 1871,
  'teamID': 'FW1',
  'lgID': None,
  'inseason': 2,
  'G': 5,
  'W': 2,
  'L': 3,
  'rank': 8,
  'plyrMgr': 'Y',
  'id': 4}]

## Part 2 - Awards for Managers

Now add a table for the `AwardsManagers.csv` table.

In [20]:
Awards = pd.read_csv('~/Desktop/baseball/core/AwardsManagers.csv')
Awards.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [21]:
Awards.columns

Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')

In [22]:
[(col, Awards[col].is_unique) for col in Awards]

[('playerID', False),
 ('awardID', False),
 ('yearID', False),
 ('lgID', False),
 ('tie', False),
 ('notes', False)]

In [23]:
Awards.shape

(179, 6)

In [24]:
Awards.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [25]:
from dfply import mutate
Awards = (Awards >>
            mutate(id = Awards.index))

In [26]:
Awards.id.is_unique

True

In [27]:
Awards.dtypes

playerID    object
awardID     object
yearID       int64
lgID        object
tie         object
notes       object
id           int64
dtype: object

In [28]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'awardID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'LgID':String, 
             'tie':String, 
             'notes':String, 
             'id':Integer}

In [29]:
from sqlalchemy import create_engine
schema = pd.io.sql.get_schema(Awards, 'awards', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE awards (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT awards_pk PRIMARY KEY (id)
)




In [30]:
mang_eng.execute(schema)

2019-01-30 12:31:41,935 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE awards (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT awards_pk PRIMARY KEY (id)
)


2019-01-30 12:31:41,936 INFO sqlalchemy.engine.base.Engine ()
2019-01-30 12:31:41,958 INFO sqlalchemy.engine.base.Engine COMMIT


In [31]:
Awards.shape

(179, 7)

In [32]:
Awards.to_sql('awards', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-30 12:36:57,858 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("awards")
2019-01-30 12:36:57,860 INFO sqlalchemy.engine.base.Engine ()
2019-01-30 12:36:57,863 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-30 12:36:57,868 INFO sqlalchemy.engine.base.Engine INSERT INTO awards ("playerID", "awardID", "yearID", "lgID", tie, notes, id) VALUES (?, ?, ?, ?, ?, ?, ?)
2019-01-30 12:36:57,869 INFO sqlalchemy.engine.base.Engine (('larusto01', 'BBWAA Manager of the Year', 1983, 'AL', None, None, 0), ('lasorto01', 'BBWAA Manager of the Year', 1983, 'NL', None, None, 1), ('andersp01', 'BBWAA Manager of the Year', 1984, 'AL', None, None, 2), ('freyji99', 'BBWAA Manager of the Year', 1984, 'NL', None, None, 3), ('coxbo01', 'BBWAA Manager of the Year', 1985, 'AL', None, None, 4), ('herzowh01', 'BBWAA Manager of the Year', 1985, 'NL', None, None, 5), ('mcnamjo99', 'BBWAA Manager of the Year', 1986, 'AL', None, None, 6), ('lanieha01', 'BBWAA Manager of the Year', 1986, 'NL', N

In [33]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [35]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Award = Base.classes.awards

In [36]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Award)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-30 12:38:11,054 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-30 12:38:11,056 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM awards
2019-01-30 12:38:11,057 INFO sqlalchemy.engine.base.Engine ()


[{'playerID': 'larusto01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1983,
  'lgID': 'AL',
  'tie': None,
  'notes': None,
  'id': 0},
 {'playerID': 'lasorto01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1983,
  'lgID': 'NL',
  'tie': None,
  'notes': None,
  'id': 1},
 {'playerID': 'andersp01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1984,
  'lgID': 'AL',
  'tie': None,
  'notes': None,
  'id': 2},
 {'playerID': 'freyji99',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1984,
  'lgID': 'NL',
  'tie': None,
  'notes': None,
  'id': 3},
 {'playerID': 'coxbo01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1985,
  'lgID': 'AL',
  'tie': None,
  'notes': None,
  'id': 4}]

## Part 3 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


In [1]:
import pandas as pd
powers = pd.read_csv('./data/super_hero_powers.csv', na_values=['-',' ', '-99.0'])
powers.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [2]:
powers.columns

Index(['hero_names', 'Agility', 'Accelerated Healing', 'Lantern Power Ring',
       'Dimensional Awareness', 'Cold Resistance', 'Durability', 'Stealth',
       'Energy Absorption', 'Flight',
       ...
       'Web Creation', 'Reality Warping', 'Odin Force', 'Symbiote Costume',
       'Speed Force', 'Phoenix Force', 'Molecular Dissipation',
       'Vision - Cryo', 'Omnipresent', 'Omniscient'],
      dtype='object', length=168)

In [3]:
powers.hero_names.is_unique

True

In [4]:
powers.dtypes

hero_names               object
Agility                    bool
Accelerated Healing        bool
Lantern Power Ring         bool
Dimensional Awareness      bool
Cold Resistance            bool
Durability                 bool
Stealth                    bool
Energy Absorption          bool
Flight                     bool
Danger Sense               bool
Underwater breathing       bool
Marksmanship               bool
Weapons Master             bool
Power Augmentation         bool
Animal Attributes          bool
Longevity                  bool
Intelligence               bool
Super Strength             bool
Cryokinesis                bool
Telepathy                  bool
Energy Armor               bool
Energy Blasts              bool
Duplication                bool
Size Changing              bool
Density Control            bool
Stamina                    bool
Astral Travel              bool
Audio Control              bool
Dexterity                  bool
                          ...  
Intuitiv

In [5]:
from dfply import *
clean_powers = lambda pow: pow.strip().lower().replace(' ', '_').replace('-','')
new_powers = {clean_powers(old_pow):old_pow for old_pow in powers.columns}
new_powers

{'hero_names': 'hero_names',
 'agility': 'Agility',
 'accelerated_healing': 'Accelerated Healing',
 'lantern_power_ring': 'Lantern Power Ring',
 'dimensional_awareness': 'Dimensional Awareness',
 'cold_resistance': 'Cold Resistance',
 'durability': 'Durability',
 'stealth': 'Stealth',
 'energy_absorption': 'Energy Absorption',
 'flight': 'Flight',
 'danger_sense': 'Danger Sense',
 'underwater_breathing': 'Underwater breathing',
 'marksmanship': 'Marksmanship',
 'weapons_master': 'Weapons Master',
 'power_augmentation': 'Power Augmentation',
 'animal_attributes': 'Animal Attributes',
 'longevity': 'Longevity',
 'intelligence': 'Intelligence',
 'super_strength': 'Super Strength',
 'cryokinesis': 'Cryokinesis',
 'telepathy': 'Telepathy',
 'energy_armor': 'Energy Armor',
 'energy_blasts': 'Energy Blasts',
 'duplication': 'Duplication',
 'size_changing': 'Size Changing',
 'density_control': 'Density Control',
 'stamina': 'Stamina',
 'astral_travel': 'Astral Travel',
 'audio_control': 'Audio

In [6]:
from dfply import rename
powers_rebrand = (powers >>
                 rename(**new_powers))
powers_rebrand.columns

Index(['hero_names', 'agility', 'accelerated_healing', 'lantern_power_ring',
       'dimensional_awareness', 'cold_resistance', 'durability', 'stealth',
       'energy_absorption', 'flight',
       ...
       'web_creation', 'reality_warping', 'odin_force', 'symbiote_costume',
       'speed_force', 'phoenix_force', 'molecular_dissipation', 'vision__cryo',
       'omnipresent', 'omniscient'],
      dtype='object', length=168)

## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

In [44]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///databases/super_powers.db', echo=False)

In [45]:
from sqlalchemy import String, Boolean
sql_types = {'hero_names': String,
             'agility': Boolean,
             'accelerated_healing': Boolean,
             'lantern_power_ring': Boolean,
             'dimensional_awareness': Boolean,
             'cold_resistance': Boolean,
             'durability': Boolean,
             'stealth': Boolean,
             'energy_absorption': Boolean,
             'flight': Boolean,
             'danger_sense': Boolean,
             'underwater_breathing':Boolean,
             'marksmanship': Boolean,
             'weapons_master': Boolean,
             'power_augmentation': Boolean,
             'animal_attributes': Boolean,
             'longevity': Boolean,
             'intelligence': Boolean,
             'super_strength': Boolean,
             'cryokinesis': Boolean,
             'telepathy': Boolean,
             'energy_armor': Boolean,
             'energy_blasts': Boolean,
             'duplication': Boolean,
             'size_changing': Boolean,
             'density_control': Boolean,
             'stamina': Boolean,
             'astral_travel': Boolean,
             'audio_control': Boolean,
             'dexterity': Boolean,
             'omnitrix': Boolean,
             'super_speed': Boolean,
             'possession': Boolean,
             'animal_oriented_powers': Boolean,
             'weaponbased_powers': Boolean,
             'electrokinesis': Boolean,
             'darkforce_manipulation': Boolean,
             'death_touch': Boolean,
             'teleportation': Boolean,
             'enhanced_senses': Boolean,
             'telekinesis': Boolean,
             'energy_beams': Boolean,
             'magic': Boolean,
             'hyperkinesis': Boolean,
             'jump': Boolean,
             'clairvoyance': Boolean,
             'dimensional_travel': Boolean,
             'power_sense': Boolean,
             'shapeshifting': Boolean,
             'peak_human_condition': Boolean,
             'immortality': Boolean,
             'camouflage': Boolean,
             'element_control': Boolean,
             'phasing': Boolean,
             'astral_projection':Boolean,
             'electrical_transport': Boolean,
             'fire_control': Boolean,
             'projection': Boolean,
             'summoning': Boolean,
             'enhanced_memory': Boolean,
             'reflexes': Boolean,
             'invulnerability': Boolean,
             'energy_constructs': Boolean,
             'force_fields': Boolean,
             'selfsustenance': Boolean,
             'antigravity': Boolean,
             'empathy': Boolean,
             'power_nullifier': Boolean,
             'radiation_control': Boolean,
             'psionic_powers': Boolean,
             'elasticity': Boolean,
             'substance_secretion': Boolean,
             'elemental_transmogrification': Boolean,
             'technopath/cyberpath': Boolean,
             'photographic_reflexes': Boolean,
             'seismic_power': Boolean,
             'animation': Boolean,
             'precognition': Boolean,
             'mind_control': Boolean,
             'fire_resistance': Boolean,
             'power_absorption': Boolean,
             'enhanced_hearing': Boolean,
             'nova_force': Boolean,
             'insanity': Boolean,
             'hypnokinesis': Boolean,
             'animal_control': Boolean,
             'natural_armor': Boolean,
             'intangibility': Boolean,
             'enhanced_sight': Boolean,
             'molecular_manipulation': Boolean,
             'heat_generation': Boolean,
             'adaptation': Boolean,
             'gliding': Boolean,
             'power_suit': Boolean,
             'mind_blast': Boolean,
             'probability_manipulation': Boolean,
             'gravity_control': Boolean,
             'regeneration': Boolean,
             'light_control': Boolean,
             'echolocation': Boolean,
             'levitation': Boolean,
             'toxin_and_disease_control': Boolean,
             'banish': Boolean,
             'energy_manipulation': Boolean,
             'heat_resistance': Boolean,
             'natural_weapons': Boolean,
             'time_travel': Boolean,
             'enhanced_smell': Boolean,
             'illusions': Boolean,
             'thirstokinesis': Boolean,
             'hair_manipulation': Boolean,
             'illumination': Boolean,
             'omnipotent': Boolean,
             'cloaking': Boolean,
             'changing_armor': Boolean,
             'power_cosmic': Boolean,
             'biokinesis': Boolean,
             'water_control': Boolean,
             'radiation_immunity': Boolean,
             'vision__telescopic': Boolean,
             'toxin_and_disease_resistance': Boolean,
             'spatial_awareness': Boolean,
             'energy_resistance': Boolean,
             'telepathy_resistance': Boolean,
             'molecular_combustion': Boolean,
             'omnilingualism': Boolean,
             'portal_creation': Boolean,
             'magnetism': Boolean,
             'mind_control_resistance': Boolean,
             'plant_control': Boolean,
             'sonar': Boolean,
             'sonic_scream': Boolean,
             'time_manipulation': Boolean,
             'enhanced_touch': Boolean,
             'magic_resistance': Boolean,
             'invisibility': Boolean,
             'submariner': Boolean,
             'radiation_absorption': Boolean,
             'intuitive_aptitude': Boolean,
             'vision__microscopic': Boolean,
             'melting': Boolean,
             'wind_control': Boolean,
             'super_breath': Boolean,
             'wallcrawling': Boolean,
             'vision__night': Boolean,
             'vision__infrared': Boolean,
             'grim_reaping': Boolean,
             'matter_absorption': Boolean,
             'the_force': Boolean,
             'resurrection': Boolean,
             'terrakinesis': Boolean,
             'vision__heat': Boolean,
             'vitakinesis': Boolean,
             'radar_sense': Boolean,
             'qwardian_power_ring': Boolean,
             'weather_control': Boolean,
             'vision__xray': Boolean,
             'vision__thermal': Boolean,
             'web_creation': Boolean,
             'reality_warping': Boolean,
             'odin_force': Boolean,
             'symbiote_costume': Boolean,
             'speed_force': Boolean,
             'phoenix_force': Boolean,
             'molecular_dissipation': Boolean,
             'vision__cryo': Boolean,
             'omnipresent': Boolean,
             'omniscient': Boolean}


In [46]:
schema = pd.io.sql.get_schema(powers_rebrand, # dataframe
                              'super_powers', # name in SQL db
                              keys='hero_names', # primary key
                              con=engine, # connection
                              dtype=sql_types # SQL types
                             )
print(schema)


CREATE TABLE super_powers (
	hero_names VARCHAR NOT NULL, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armor BOOLEAN, 
	energy_blasts BOOLEAN, 
	duplication BOOLEAN, 
	size_changing BOOLEAN, 
	density_control BOOLEAN, 
	stamina BOOLEAN, 
	astral_travel BOOLEAN, 
	audio_control BOOLEAN, 
	dexterity BOOLEAN, 
	omnitrix BOOLEAN, 
	super_speed BOOLEAN, 
	possession BOOLEAN, 
	animal_oriented_powers BOOLEAN, 
	weaponbased_powers BOOLEAN, 
	electrokinesis BOOLEAN, 
	darkforce_manipulation BOOLEAN, 
	death_touch BOOLEAN, 
	telep

In [47]:
engine.execute(schema)

In [48]:
powers_rebrand.to_sql('super_powers', 
                  con=engine, 
                  dtype=sql_types, 
                  index=False,
                  if_exists='append')

## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

In [49]:
from sqlalchemy import create_engine
engine2 = create_engine('sqlite:///databases/super_powers.db')

In [50]:
engine2.echo = False

In [52]:
from sqlalchemy.ext.automap import automap_base

Base = automap_base()
engine2 = create_engine("sqlite:///databases/super_powers.db")
Base.prepare(engine2, reflect=True)

Powers = Base.classes.super_powers

In [54]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine2)
session = Session()

In [56]:
from sqlalchemy import inspect
insp = inspect(engine2)

In [57]:
insp.get_table_names()

['super_powers']

In [61]:
insp.get_columns('super_powers')

[{'name': 'hero_names',
  'type': VARCHAR(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'agility',
  'type': BOOLEAN(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'accelerated_healing',
  'type': BOOLEAN(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'lantern_power_ring',
  'type': BOOLEAN(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'dimensional_awareness',
  'type': BOOLEAN(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'cold_resistance',
  'type': BOOLEAN(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'durability',
  'type': BOOLEAN(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'stealth',
  'type': BOOLEAN(),
  '

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

In [90]:
from sqlalchemy import select

In [146]:
from sqlalchemy import select
# Make an SQL expression
q1_stmt = (select_sql([Powers.hero_names, 
                       Powers.super_strength, 
                       Powers.super_speed]).
            where(Powers.super_strength == True).
            where(Powers.super_speed == True)
          )
# Excute the expression
print(q1_stmt)
#ans_one = session.execute(q1_stmt).fetchall()
result = pd.read_sql_query(ff_stmt, con=engine)
result

SELECT super_powers.hero_names, super_powers.super_strength, super_powers.super_speed 
FROM super_powers 
WHERE super_powers.super_strength = true AND super_powers.super_speed = true


,hero_names,super_strength,super_speed
0,3-D Man,True,True
1,Abomination,True,True
2,Abraxas,True,True
3,Air-Walker,True,True
4,Ajax,True,True
5,Alex Mercer,True,True
6,Alien,True,True
7,Amazo,True,True
8,Angel,True,True
9,Angel Dust,True,True


## Answer to Question 1:
                    219 Heroes have both Super Strength and Super Speed

In [160]:
q2_stmt = (select_sql([Powers.hero_names]).
           where(Powers.hero_names = 'black_'))
# Excute the expression
result2 =pd.read_sql_query(q2_stmt, con=engine)

SyntaxError: keyword can't be an expression (<ipython-input-160-f07c4a4e104f>, line 2)

In [162]:
#result2

In [102]:
#TOTAL QUESTION TWO UP

In [167]:
q31_stmt = (select_sql([Powers.hero_names, 
                       Powers.agility, 
                       Powers.stealth]).
            where(Powers.agility == True))
# Excute the expression
result31 = pd.read_sql_query(q31_stmt, con=engine)
result31

,hero_names,agility,stealth
0,3-D Man,True,False
1,Abe Sapien,True,False
2,Ajax,True,False
3,Alex Mercer,True,True
4,Alien,True,True
5,Amazo,True,False
6,Angel,True,True
7,Angel Dust,True,False
8,Annihilus,True,False
9,Ant-Man II,True,True


In [168]:
q32_stmt = (select_sql([Powers.hero_names, 
                       Powers.agility, 
                       Powers.stealth]).
            where(Powers.agility == True).
            where(Powers.stealth == True))
# Excute the expression
result32 = pd.read_sql_query(q32_stmt, con=engine)
result32

,hero_names,agility,stealth
0,Alex Mercer,True,True
1,Alien,True,True
2,Angel,True,True
3,Ant-Man II,True,True
4,Aquaman,True,True
5,Batgirl IV,True,True
6,Batman,True,True
7,Batman II,True,True
8,Black Canary,True,True
9,Black Flash,True,True


In [170]:
Q3_total = 242
Q3_no_stealth = 242-95
Q3_stealth = 95

per_stealth = Q3_stealth/Q3_total
per_nostealth = Q3_no_stealth/Q3_total

(per_stealth, per_nostealth)

(0.3925619834710744, 0.6074380165289256)

## Answer to Question 3:
          No, heroes with Agility are not more likely to have stealth, those with Agility have a 39.25% chance of having stealth while those without Agility have a 60.74% chance of having stealth

In [171]:
q41_stmt = (select_sql([Powers.hero_names, 
                       Powers.flight, 
                       Powers.super_strength]).
            where(Powers.flight == True))
# Excute the expression
result41 = pd.read_sql_query(q41_stmt, con=engine)
result41

,hero_names,flight,super_strength
0,Abraxas,True,True
1,Adam Strange,True,False
2,Air-Walker,True,True
3,Alan Scott,True,False
4,Amazo,True,True
5,Angel,True,True
6,Angel Salvadore,True,False
7,Annihilus,True,True
8,Anti-Monitor,True,True
9,Apocalypse,True,True


In [173]:
q42_stmt = (select_sql([Powers.hero_names, 
                       Powers.flight, 
                       Powers.super_strength]).
            where(Powers.flight == True).
            where(Powers.super_strength == True))
# Excute the expression
result = pd.read_sql_query(q42_stmt, con=engine)
result

,hero_names,flight,super_strength
0,Abraxas,True,True
1,Air-Walker,True,True
2,Amazo,True,True
3,Angel,True,True
4,Annihilus,True,True
5,Anti-Monitor,True,True
6,Apocalypse,True,True
7,Archangel,True,True
8,Ardina,True,True
9,Astro Boy,True,True


In [174]:
Q4_total = 212
Q4_strength = 147
answ4 = Q4_strength/Q4_total
answ4

0.6933962264150944

## Answer to Question 4:
        The fraction of all heroes that can fly and also have Super Strength is 147/212 or 69.3%

In [157]:
q5_stmt = (select_sql([Powers.hero_names]).
            where(Powers.hero_names.startswith('boy')).
            limit(5))
# Excute the expression
result = pd.read_sql_query(q5_stmt, con=engine)
result

,hero_names


## Problem 5

Tell me another cool fact about the super powers.

In [177]:
q5_stmt = (select_sql([Powers.hero_names, 
                       Powers.audio_control,
                       Powers.enhanced_hearing,]).
            where(Powers.audio_control == True))
# Excute the expression
result5 = pd.read_sql_query(q5_stmt, con=engine)

In [178]:
result5

,hero_names,audio_control,enhanced_hearing
0,Banshee,True,True
1,Black Canary,True,False
2,Dazzler,True,False
3,Klaw,True,False
4,Red Tornado,True,True
5,Shriek,True,False
6,Songbird,True,False


# Answer to Problem 5:
        There are out of the 6 heroes that have control over audio only 2 also have enhanced_hearing